In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import logging

In [2]:
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [3]:
base_path = './data'
feature_path = './feature'

In [4]:
# 加载邀请回答数据
train = pd.read_csv(f'{base_path}/invite_info_0926.txt', sep='\t', header=None)
train.columns = ['qid', 'uid', 'dt', 'label']
del train['dt']
logging.info("invite %s", train.shape)

test = pd.read_csv(f'{base_path}/invite_info_evaluate_0926.txt', sep='\t', header=None)
test.columns = ['qid', 'uid', 'dt']
sub = test.copy()
sub_size = len(sub)
del test['dt']
logging.info("test %s", test.shape)

[2019-12-07 03:03:11,981] INFO in <ipython-input-4-f5fcce844f4e>: invite (9489162, 3)
[2019-12-07 03:03:13,532] INFO in <ipython-input-4-f5fcce844f4e>: test (1141683, 2)


In [5]:
# 加载 kfold feature
t1 = pd.read_csv(f'{feature_path}/train_kfold_feature.txt', sep='\t')
train = pd.concat([train, t1], axis=1)

t1 = pd.read_csv(f'{feature_path}/test_kfold_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)

In [6]:
# 加载 user 过去两个月的回答统计特征（除当条记录）
# t1 = pd.read_csv(f'{feature_path}/train_ua_feature.txt', sep='\t')
# train = pd.concat([train, t1], axis=1)

# t1 = pd.read_csv(f'{feature_path}/test_ua_feature.txt', sep='\t')
# test = pd.concat([test, t1], axis=1)

In [7]:
# 加载 invete feature 1
t1 = pd.read_csv(f'{feature_path}/train_invite_feature.txt', sep='\t')
train = pd.concat([train, t1], axis=1)

t1 = pd.read_csv(f'{feature_path}/test_invite_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)

In [8]:
# 加载 invete feature 2
t1 = pd.read_csv(f'{feature_path}/train_invite_feature_2.txt', sep='\t')
train = pd.concat([train, t1], axis=1)

t1 = pd.read_csv(f'{feature_path}/test_invite_feature_2.txt', sep='\t')
test = pd.concat([test, t1], axis=1)

In [9]:
# 加载 kfold topic feature
t1 = pd.read_csv(f'{feature_path}/train_kfold_topic_feature.txt', sep='\t')
train = pd.concat([train, t1], axis=1)

t1 = pd.read_csv(f'{feature_path}/test_kfold_topic_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)

In [10]:
# 加载 topic feature
# t1 = pd.read_csv(f'{feature_path}/train_topic_feature.txt', sep='\t')
# train = pd.concat([train, t1], axis=1)

# t1 = pd.read_csv(f'{feature_path}/test_topic_feature.txt', sep='\t')
# test = pd.concat([test, t1], axis=1)

In [11]:
# 加载用户
user = pd.read_csv(f'{base_path}/member_info_0926.txt', header=None, sep='\t')
user.columns = ['uid', 'gender', 'freq', 'uf_b1', 'uf_b2','uf_b3', 'uf_b4', 'uf_b5', 
                'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5',  'score', 'follow_topic', 'inter_topic']

del user['follow_topic'], user['inter_topic']
logging.info("user %s", user.shape)

unq = user.nunique()
logging.info("user unq %s", unq)

for x in unq[unq == 1].index:
    del user[x]
    logging.info('del unq==1 %s', x)

t = user.dtypes
cats = [x for x in t[t == 'object'].index if x not in ['follow_topic', 'inter_topic', 'uid']]
logging.info("user cat %s", cats)

for d in cats:
    lb = LabelEncoder()
    user[d] = lb.fit_transform(user[d])
    logging.info('encode %s', d)
    
q_lb = LabelEncoder()
q_lb.fit(list(train['qid'].astype(str).values) + list(test['qid'].astype(str).values))
train['qid_enc'] = q_lb.transform(train['qid'])
test['qid_enc'] = q_lb.transform(test['qid'])

u_lb = LabelEncoder()
u_lb.fit(user['uid'])
train['uid_enc'] = u_lb.transform(train['uid'])
test['uid_enc'] = u_lb.transform(test['uid'])

# merge user
train = pd.merge(train, user, on='uid', how='left')
test = pd.merge(test, user, on='uid', how='left')
logging.info("train shape %s, test shape %s", train.shape, test.shape)

[2019-12-07 03:07:12,066] INFO in <ipython-input-11-af285af762e4>: user (1931654, 14)
[2019-12-07 03:07:15,524] INFO in <ipython-input-11-af285af762e4>: user unq uid       1931654
gender          3
freq            5
uf_b1           2
uf_b2           2
uf_b3           2
uf_b4           2
uf_b5           2
uf_c1        2561
uf_c2         291
uf_c3         428
uf_c4        1556
uf_c5           2
score         732
dtype: int64
[2019-12-07 03:07:15,529] INFO in <ipython-input-11-af285af762e4>: user cat ['gender', 'freq', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
[2019-12-07 03:07:16,396] INFO in <ipython-input-11-af285af762e4>: encode gender
[2019-12-07 03:07:17,274] INFO in <ipython-input-11-af285af762e4>: encode freq
[2019-12-07 03:07:18,103] INFO in <ipython-input-11-af285af762e4>: encode uf_c1
[2019-12-07 03:07:18,912] INFO in <ipython-input-11-af285af762e4>: encode uf_c2
[2019-12-07 03:07:19,725] INFO in <ipython-input-11-af285af762e4>: encode uf_c3
[2019-12-07 03:07:20,513] INFO in

In [12]:
data = pd.concat((train, test), axis=0, sort=True)
len_train = len(train)
del train

In [14]:
# count 特征
count_fea = ['uid_enc', 'qid_enc', 'gender', 'freq', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
for feat in count_fea:
    col_name = '{}_count'.format(feat)
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data.loc[data[col_name] < 2, feat] = -1
    data[feat] += 1
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data[col_name] = (data[col_name] - data[col_name].min()) / (data[col_name].max() - data[col_name].min())

In [15]:
data['wk'] = data['day'] % 7

In [16]:
drop_feat = ('label', 'uid', 'qid', 'dt', 'day') 
# drop_feat += ('q_ans_kfold_count', 'q_diff_qa_days_max', 'q_diff_qa_days_mean', 'q_diff_qa_days_sum', 
#               'q_has_img_max', 'q_has_img_mean', 'q_has_img_sum', 'q_has_video_max', 'q_has_video_mean', 
#               'q_has_video_sum','q_is_dest_max', 'q_is_dest_mean', 'q_is_dest_sum', 'q_is_good_max', 
#               'q_is_good_mean', 'q_is_good_sum', 'q_is_rec_max', 'q_is_rec_mean', 'q_is_rec_sum', 
#               'q_reci_cheer_max', 'q_reci_cheer_mean', 'q_reci_cheer_sum', 'q_reci_comment_max', 
#               'q_reci_comment_mean', 'q_reci_comment_sum', 'q_reci_dis_max', 'q_reci_dis_mean', 
#               'q_reci_dis_sum', 'q_reci_mark_max', 'q_reci_mark_mean', 'q_reci_mark_sum', 'q_reci_no_help_max',
#               'q_reci_no_help_mean', 'q_reci_no_help_sum', 'q_reci_tks_max', 'q_reci_tks_mean', 
#               'q_reci_tks_sum', 'q_reci_uncheer_max', 'q_reci_uncheer_mean', 'q_reci_uncheer_sum', 
#               'q_reci_xxx_max', 'q_reci_xxx_mean', 'q_reci_xxx_sum', 'q_word_count_max', 'q_word_count_mean', 
#               'q_word_count_sum')
# drop_feat += ('u_ans_kfold_count', 'u_diff_qa_days_max', 'u_diff_qa_days_mean', 'u_diff_qa_days_sum', 
#               'u_has_img_max', 'u_has_img_mean', 'u_has_img_sum', 'u_has_video_max', 'u_has_video_mean', 
#               'u_has_video_sum', 'u_is_dest_max', 'u_is_dest_mean', 'u_is_dest_sum', 'u_is_good_max', 
#               'u_is_good_mean', 'u_is_good_sum', 'u_is_rec_max', 'u_is_rec_mean', 'u_is_rec_sum', 'u_reci_cheer_max', 
#               'u_reci_cheer_mean', 'u_reci_cheer_sum', 'u_reci_comment_max', 'u_reci_comment_mean',
#               'u_reci_comment_sum', 'u_reci_dis_max', 'u_reci_dis_mean', 'u_reci_dis_sum', 'u_reci_mark_max', 
#               'u_reci_mark_mean', 'u_reci_mark_sum', 'u_reci_no_help_max', 'u_reci_no_help_mean', 
#               'u_reci_no_help_sum', 'u_reci_tks_max', 'u_reci_tks_mean', 'u_reci_tks_sum', 'u_reci_uncheer_max', 
#               'u_reci_uncheer_mean', 'u_reci_uncheer_sum', 'u_reci_xxx_max', 'u_reci_xxx_mean', 'u_reci_xxx_sum', 
#               'u_word_count_max', 'u_word_count_mean', 'u_word_count_sum')
# drop_feat += ('u_total_answer',)
feature_cols = [x for x in data.columns if x not in drop_feat]
# feature_cols

In [23]:
logging.info("feature size %s", len(feature_cols))

X_train_all = data.iloc[:len_train][feature_cols]
y_train_all = data.iloc[:len_train]['label']
X_test = data.iloc[len_train:]
assert len(X_test) == sub_size

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    break

X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]
del X_train_all

logging.info("train shape %s, val shape %s, test shape %s", X_train.shape, X_val.shape, X_test.shape)

model_lgb = LGBMClassifier(n_estimators=5000, n_jobs=-1, objective='binary', seed=1000, silent=True)
model_lgb.fit(X_train, y_train,
              eval_metric=['logloss', 'auc'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=50)



[2019-12-07 05:40:56,179] INFO in <ipython-input-23-e1b5360c1f33>: feature size 196
[2019-12-07 05:49:44,289] INFO in <ipython-input-23-e1b5360c1f33>: train shape (7591329, 196), val shape (1897833, 196), test shape (1141683, 200)


[1]	valid_0's auc: 0.759659	valid_0's binary_logloss: 0.452689
Training until validation scores don't improve for 50 rounds
[2]	valid_0's auc: 0.779539	valid_0's binary_logloss: 0.441224
[3]	valid_0's auc: 0.791482	valid_0's binary_logloss: 0.431569
[4]	valid_0's auc: 0.796117	valid_0's binary_logloss: 0.423891
[5]	valid_0's auc: 0.801523	valid_0's binary_logloss: 0.416566
[6]	valid_0's auc: 0.803384	valid_0's binary_logloss: 0.41089
[7]	valid_0's auc: 0.805883	valid_0's binary_logloss: 0.405654
[8]	valid_0's auc: 0.808221	valid_0's binary_logloss: 0.400678
[9]	valid_0's auc: 0.810074	valid_0's binary_logloss: 0.396834
[10]	valid_0's auc: 0.812767	valid_0's binary_logloss: 0.392862
[11]	valid_0's auc: 0.814499	valid_0's binary_logloss: 0.389612
[12]	valid_0's auc: 0.815411	valid_0's binary_logloss: 0.386634
[13]	valid_0's auc: 0.817139	valid_0's binary_logloss: 0.383691
[14]	valid_0's auc: 0.818227	valid_0's binary_logloss: 0.381261
[15]	valid_0's auc: 0.819354	valid_0's binary_logloss

[129]	valid_0's auc: 0.856544	valid_0's binary_logloss: 0.330548
[130]	valid_0's auc: 0.856616	valid_0's binary_logloss: 0.330458
[131]	valid_0's auc: 0.856723	valid_0's binary_logloss: 0.330345
[132]	valid_0's auc: 0.856864	valid_0's binary_logloss: 0.330193
[133]	valid_0's auc: 0.856978	valid_0's binary_logloss: 0.330087
[134]	valid_0's auc: 0.8571	valid_0's binary_logloss: 0.329948
[135]	valid_0's auc: 0.857169	valid_0's binary_logloss: 0.329878
[136]	valid_0's auc: 0.857294	valid_0's binary_logloss: 0.329721
[137]	valid_0's auc: 0.857374	valid_0's binary_logloss: 0.329637
[138]	valid_0's auc: 0.85748	valid_0's binary_logloss: 0.329542
[139]	valid_0's auc: 0.857607	valid_0's binary_logloss: 0.329401
[140]	valid_0's auc: 0.857677	valid_0's binary_logloss: 0.329327
[141]	valid_0's auc: 0.857768	valid_0's binary_logloss: 0.329243
[142]	valid_0's auc: 0.857851	valid_0's binary_logloss: 0.329162
[143]	valid_0's auc: 0.857924	valid_0's binary_logloss: 0.329086
[144]	valid_0's auc: 0.85803

[256]	valid_0's auc: 0.864846	valid_0's binary_logloss: 0.321775
[257]	valid_0's auc: 0.864893	valid_0's binary_logloss: 0.321732
[258]	valid_0's auc: 0.864926	valid_0's binary_logloss: 0.321681
[259]	valid_0's auc: 0.864969	valid_0's binary_logloss: 0.32162
[260]	valid_0's auc: 0.865015	valid_0's binary_logloss: 0.321574
[261]	valid_0's auc: 0.865053	valid_0's binary_logloss: 0.321537
[262]	valid_0's auc: 0.865079	valid_0's binary_logloss: 0.321506
[263]	valid_0's auc: 0.865105	valid_0's binary_logloss: 0.32148
[264]	valid_0's auc: 0.865124	valid_0's binary_logloss: 0.321459
[265]	valid_0's auc: 0.865168	valid_0's binary_logloss: 0.321413
[266]	valid_0's auc: 0.865201	valid_0's binary_logloss: 0.32138
[267]	valid_0's auc: 0.865235	valid_0's binary_logloss: 0.321338
[268]	valid_0's auc: 0.865281	valid_0's binary_logloss: 0.321286
[269]	valid_0's auc: 0.865322	valid_0's binary_logloss: 0.321246
[270]	valid_0's auc: 0.865357	valid_0's binary_logloss: 0.321209
[271]	valid_0's auc: 0.86538

[383]	valid_0's auc: 0.868638	valid_0's binary_logloss: 0.317693
[384]	valid_0's auc: 0.868653	valid_0's binary_logloss: 0.317678
[385]	valid_0's auc: 0.868675	valid_0's binary_logloss: 0.317658
[386]	valid_0's auc: 0.868687	valid_0's binary_logloss: 0.317645
[387]	valid_0's auc: 0.868708	valid_0's binary_logloss: 0.317622
[388]	valid_0's auc: 0.868724	valid_0's binary_logloss: 0.317602
[389]	valid_0's auc: 0.868741	valid_0's binary_logloss: 0.317581
[390]	valid_0's auc: 0.868774	valid_0's binary_logloss: 0.31755
[391]	valid_0's auc: 0.868795	valid_0's binary_logloss: 0.31753
[392]	valid_0's auc: 0.868819	valid_0's binary_logloss: 0.317504
[393]	valid_0's auc: 0.868836	valid_0's binary_logloss: 0.317485
[394]	valid_0's auc: 0.868854	valid_0's binary_logloss: 0.317461
[395]	valid_0's auc: 0.86886	valid_0's binary_logloss: 0.317456
[396]	valid_0's auc: 0.868873	valid_0's binary_logloss: 0.317442
[397]	valid_0's auc: 0.868906	valid_0's binary_logloss: 0.317406
[398]	valid_0's auc: 0.86893

...

[3393]	valid_0's auc: 0.880606	valid_0's binary_logloss: 0.304139
[3394]	valid_0's auc: 0.880607	valid_0's binary_logloss: 0.304138
[3395]	valid_0's auc: 0.880609	valid_0's binary_logloss: 0.304137
[3396]	valid_0's auc: 0.880609	valid_0's binary_logloss: 0.304136
[3397]	valid_0's auc: 0.88061	valid_0's binary_logloss: 0.304136
[3398]	valid_0's auc: 0.88061	valid_0's binary_logloss: 0.304136
[3399]	valid_0's auc: 0.88061	valid_0's binary_logloss: 0.304136
[3400]	valid_0's auc: 0.88061	valid_0's binary_logloss: 0.304135
[3401]	valid_0's auc: 0.88061	valid_0's binary_logloss: 0.304135
[3402]	valid_0's auc: 0.88061	valid_0's binary_logloss: 0.304135
[3403]	valid_0's auc: 0.880611	valid_0's binary_logloss: 0.304132
[3404]	valid_0's auc: 0.880616	valid_0's binary_logloss: 0.304128
[3405]	valid_0's auc: 0.880617	valid_0's binary_logloss: 0.304128
[3406]	valid_0's auc: 0.880619	valid_0's binary_logloss: 0.304125
[3407]	valid_0's auc: 0.88062	valid_0's binary_logloss: 0.304124
[3408]	valid_0's 

[3518]	valid_0's auc: 0.880767	valid_0's binary_logloss: 0.30396
[3519]	valid_0's auc: 0.880769	valid_0's binary_logloss: 0.303959
[3520]	valid_0's auc: 0.880768	valid_0's binary_logloss: 0.303959
[3521]	valid_0's auc: 0.880768	valid_0's binary_logloss: 0.303959
[3522]	valid_0's auc: 0.880769	valid_0's binary_logloss: 0.303958
[3523]	valid_0's auc: 0.88077	valid_0's binary_logloss: 0.303957
[3524]	valid_0's auc: 0.88077	valid_0's binary_logloss: 0.303956
[3525]	valid_0's auc: 0.880771	valid_0's binary_logloss: 0.303955
[3526]	valid_0's auc: 0.880771	valid_0's binary_logloss: 0.303954
[3527]	valid_0's auc: 0.880773	valid_0's binary_logloss: 0.303952
[3528]	valid_0's auc: 0.880775	valid_0's binary_logloss: 0.30395
[3529]	valid_0's auc: 0.880775	valid_0's binary_logloss: 0.303951
[3530]	valid_0's auc: 0.880776	valid_0's binary_logloss: 0.303949
[3531]	valid_0's auc: 0.880776	valid_0's binary_logloss: 0.303949
[3532]	valid_0's auc: 0.880777	valid_0's binary_logloss: 0.303949
[3533]	valid_0

[3643]	valid_0's auc: 0.880899	valid_0's binary_logloss: 0.303802
[3644]	valid_0's auc: 0.880899	valid_0's binary_logloss: 0.303802
[3645]	valid_0's auc: 0.880898	valid_0's binary_logloss: 0.303802
[3646]	valid_0's auc: 0.880904	valid_0's binary_logloss: 0.303795
[3647]	valid_0's auc: 0.880904	valid_0's binary_logloss: 0.303795
[3648]	valid_0's auc: 0.880904	valid_0's binary_logloss: 0.303794
[3649]	valid_0's auc: 0.880905	valid_0's binary_logloss: 0.303794
[3650]	valid_0's auc: 0.880904	valid_0's binary_logloss: 0.303795
[3651]	valid_0's auc: 0.880905	valid_0's binary_logloss: 0.303794
[3652]	valid_0's auc: 0.880908	valid_0's binary_logloss: 0.303792
[3653]	valid_0's auc: 0.880908	valid_0's binary_logloss: 0.303791
[3654]	valid_0's auc: 0.880912	valid_0's binary_logloss: 0.303787
[3655]	valid_0's auc: 0.880914	valid_0's binary_logloss: 0.303785
[3656]	valid_0's auc: 0.880914	valid_0's binary_logloss: 0.303785
[3657]	valid_0's auc: 0.880913	valid_0's binary_logloss: 0.303786
[3658]	val

[3768]	valid_0's auc: 0.881015	valid_0's binary_logloss: 0.303668
[3769]	valid_0's auc: 0.881015	valid_0's binary_logloss: 0.303667
[3770]	valid_0's auc: 0.881016	valid_0's binary_logloss: 0.303666
[3771]	valid_0's auc: 0.881016	valid_0's binary_logloss: 0.303666
[3772]	valid_0's auc: 0.881017	valid_0's binary_logloss: 0.303665
[3773]	valid_0's auc: 0.881017	valid_0's binary_logloss: 0.303665
[3774]	valid_0's auc: 0.881017	valid_0's binary_logloss: 0.303666
[3775]	valid_0's auc: 0.881017	valid_0's binary_logloss: 0.303666
[3776]	valid_0's auc: 0.881017	valid_0's binary_logloss: 0.303666
[3777]	valid_0's auc: 0.881017	valid_0's binary_logloss: 0.303667
[3778]	valid_0's auc: 0.881017	valid_0's binary_logloss: 0.303666
[3779]	valid_0's auc: 0.881017	valid_0's binary_logloss: 0.303666
[3780]	valid_0's auc: 0.881018	valid_0's binary_logloss: 0.303666
[3781]	valid_0's auc: 0.881019	valid_0's binary_logloss: 0.303664
[3782]	valid_0's auc: 0.881019	valid_0's binary_logloss: 0.303664
[3783]	val

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=5000, n_jobs=-1, num_leaves=31, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, seed=1000,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [24]:
sub['label'] = model_lgb.predict_proba(X_test[feature_cols])[:, 1]
sub.to_csv('./result/5000_alltopic_881046.txt', index=None, header=None, sep='\t')

In [26]:
fi = pd.DataFrame({'feature': feature_cols, 'imp': model_lgb.feature_importances_})
fi['rate'] = fi['imp'] / fi['imp'].sum()
fi.sort_values(by='rate', ascending=False)[:60]

,feature,imp,rate
99,score,4120,0.035951
1,diff_iq_hour,3410,0.029756
186,uid_enc_count,2475,0.021597
181,uid_hour_std,2146,0.018726
119,u_inv_kfold_mean,2118,0.018482
81,qid_hour_count,2108,0.018394
4,hour,2047,0.017862
169,uid_diff_day_daymean,2043,0.017827
118,u_inv_kfold_count,1705,0.014878
72,qid_day_count,1656,0.014450
